# Features selection with STEPWISE

In [1]:
import sys
my_path = r'/home/ilaria/Scrivania/Machine_Learning/Project_1/Project1_ML'
sys.path.insert(0,my_path + r'/code/COMMON')

import numpy as np 
import matplotlib.pyplot as plt
from proj1_helpers import load_csv_data, predict_labels 
from implementations import least_squares
from outliers import handle_outliers
from labels import idx_2labels
from standard import standardize

In [2]:
yb, input_data, ids = load_csv_data(my_path + r'/data/train.csv', sub_sample=False)

In [3]:
input_data.shape

(250000, 30)

In [4]:
input_data, Y = handle_outliers(input_data,yb,-999,0) # substiution with zeros
ind_back, ind_sig = idx_2labels(Y, [-1,1])

input_data, mean_X, std_X = standardize(input_data)    

-999 are replaced by 0


In [5]:
# Subdived the X features space in single features
all_features = np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:]

features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))

## R^2  as stopping criteria : R2 of Tjur or R2 of McFadden

The only way to use the stepwise is using R2 of Tjur or Cox and Snell or McFadden because of the binary values of the indipendent variable

In [6]:
def results_r2_stepwise(list_r2_adj,indices_features):
    print("R2 asjusted values:")
    
    for i in range(len(list_r2_adj)):
        print(list_r2_adj[i])
    print("-------------------------------------------------------")
    print("Number of features chosen:", len(indices_features))
    print("\n")
    print("Indices of features chosen: ", indices_features)
    

### Least square model

#### Use of the error before binarization (R2 with loss)

In [7]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = least_squares(Y,X)  # The loss cannot be used as a measure for the feature selection because it's a 
y = predict_labels(w0, X)

sse = loss
sst = np.sum((Y - Y.mean())**2)  #lack of information
R2 = np.abs((sst-sse)/sst)
R2adj_0 = R2 - (k/(n-k-1)*(1-R2))

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        ws , loss = least_squares(Y,X)
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        #y = predict_labels(ws,X)   #***** NO USE OF PREDICTION
        SSE = np.sum(loss**2)
        SST = np.sum((Y- Y.mean())**2)
        R2 = np.abs((SST-SSE)/SST)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break
        

-------------------------------------------------
Feature chosen:  DER_mass_transverse_met_lep (index : 1 )
-------------------------------------------------
Feature chosen:  PRI_tau_pt (index : 13 )
-------------------------------------------------
Feature chosen:  DER_prodeta_jet_jet (index : 6 )
-------------------------------------------------
Feature chosen:  DER_met_phi_centrality (index : 11 )
-------------------------------------------------
Feature chosen:  DER_deltar_tau_lep (index : 7 )
-------------------------------------------------
Feature chosen:  DER_mass_vis (index : 2 )
-------------------------------------------------
Feature chosen:  PRI_lep_pt (index : 16 )
-------------------------------------------------
Feature chosen:  DER_pt_ratio_lep_tau (index : 10 )
-------------------------------------------------
Feature chosen:  DER_lep_eta_centrality (index : 12 )
-------------------------------------------------
Feature chosen:  PRI_met_sumet (index : 21 )
-----------

In [8]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

R2 asjusted values:
0.999999307815
0.999999361431
0.999999393469
0.999999410989
0.999999424199
0.999999436876
0.999999448652
0.999999460313
0.999999468887
0.99999947284
0.999999480841
0.999999483392
0.999999486397
0.999999488787
0.999999489339
0.999999489535
0.999999489627
0.999999489656
0.999999489683
0.999999489692
0.999999489696
0.999999489697
0.999999489698
-------------------------------------------------------
Number of features chosen: 23


Indices of features chosen:  [1, 13, 6, 11, 7, 2, 16, 10, 12, 21, 19, 5, 9, 23, 0, 26, 3, 22, 4, 18, 28, 29, 8]


#### Use of the probability of the 2 events (R2 Tjur)

In [9]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))


In [10]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = least_squares(Y,X)  # The loss cannot be used as a measure for the feature selection because it's a 
y = predict_labels(w0, X)
ind_back, ind_sig = idx_2labels(y, [-1,1])

y_ = X.dot(w0)
R2 = 0
R2adj_0 = R2 - (k/(n-k-1)*(1-R2))

#fix the R2adj_max
R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        ws , loss = least_squares(Y,X)
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws,X)          
        ind_back, ind_sig = idx_2labels(y, [-1,1])
        
        if len(ind_sig) == 0 or len(ind_back) ==0:
            print('No signal detected')
            R2_adj.append(0)
            
        else: 
            
            y_ = X.dot(ws)

            R2 = np.abs((np.mean(y_[ind_sig]) - np.mean(y_[ind_back])))
            R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
            
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)

    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen

        #idx_features.append(np.where(all_candidates[:,ind_max] == input_data))
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)

        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])

        del(X)

    else:
        break

No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
-------------------------------------------------
Feature chosen:  DER_mass_jet_jet (index : 5 )
-------------------------------------------------
Feature chosen:  PRI_jet_subleading_pt (index : 26 )
-------------------------------------------------
Feature chosen:  PRI_tau_pt (index : 13 )
-------------------------------------------------
Feature chosen:  DER_mass_vis (index : 2 )
-------------------------------------------------
Feature chosen:  DER_pt_h (index : 3 )
-------------------------------------------------
Feature chosen:  DER_sum_pt (index : 9 )
-------------------------------------------------
Feature chosen:  DER_pt_tot (index : 8 )
-------------------------------------------------
Feature c

In [11]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

R2 asjusted values:
0.711215098858
0.756230955966
0.790123036777
0.798853243572
0.806262623179
0.813879481569
0.817098685794
0.819705555408
0.820983332372
0.820994170371
0.821105661642
0.821410734028
0.821524182295
-------------------------------------------------------
Number of features chosen: 13


Indices of features chosen:  [5, 26, 13, 2, 3, 9, 8, 19, 21, 17, 6, 28, 27]


#### Use the likelihood (McFadden)

In [12]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))
   

In [13]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = least_squares(Y,X)  # The loss cannot be used as a measure for the feature selection because it's a 
y = predict_labels(w0, X)
loglike0 = np.sum(np.log(1+np.exp(X.dot(w0))) - y*(X.dot(w0)))

R2 = 0        # For the definition of McFadden 1-1 = 0
R2adj_0 = 0

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        ws , loss = least_squares(Y,X)
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws,X)   
        
        loglike = np.sum(np.log(1+np.exp(X.dot(ws))) - y*(X.dot(ws)))
        
        R2 = 1-(loglike/loglike0)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break
        

-------------------------------------------------
Feature chosen:  DER_lep_eta_centrality (index : 12 )
-------------------------------------------------
Feature chosen:  PRI_tau_pt (index : 13 )
-------------------------------------------------
Feature chosen:  DER_mass_transverse_met_lep (index : 1 )
-------------------------------------------------
Feature chosen:  DER_met_phi_centrality (index : 11 )
-------------------------------------------------
Feature chosen:  PRI_jet_num (index : 22 )
-------------------------------------------------
Feature chosen:  DER_deltaeta_jet_jet (index : 4 )
-------------------------------------------------
Feature chosen:  PRI_met (index : 19 )
-------------------------------------------------
Feature chosen:  PRI_jet_all_pt (index : 29 )
-------------------------------------------------
Feature chosen:  PRI_lep_pt (index : 16 )
-------------------------------------------------
Feature chosen:  DER_pt_h (index : 3 )
--------------------------------

In [14]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

R2 asjusted values:
0.203851601605
0.27452708355
0.357446393184
0.380909496275
0.417104747923
0.43666269786
0.442044547266
0.451721146776
0.464825449641
0.481022035899
0.486482185259
0.490548112727
0.499888932078
0.501641602113
0.502906336464
0.503627050593
0.503659349183
0.503682188862
0.503699246926
0.503711380935
0.503714720944
-------------------------------------------------------
Number of features chosen: 21


Indices of features chosen:  [12, 13, 1, 11, 22, 4, 19, 29, 16, 3, 23, 2, 7, 6, 21, 26, 18, 28, 9, 20, 14]


### Least square model using cross validation

I am not sure that cross validation can be used we don't have to estimate any hyperparameters

#### R2 with likelihood (McFadden)

In [15]:
from split_data import split_data

sys.path.insert(0,my_path + r'/code/ilaria')
from i_cross_validation_methods import *


In [16]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))

In [17]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = least_squares(Y,X)  # The loss cannot be used as a measure for the feature selection because it's a 
y = predict_labels(w0, X)
loglike0 = np.sum(np.log(1+np.exp(X.dot(w0))) - y*(X.dot(w0)))

R2 = 0        # For the definition of McFadden 1-1 = 0
R2adj_0 = 0

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        
        # CROSS-VALIDATION
        
        w_tr_tot, loss_tr_tot, loss_te_tot = cross_validation_ls(Y,X)
        ws = w_tr_tot[np.argmin(loss_te_tot)]
        
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws,X)   
        
        loglike = np.sum(np.log(1+np.exp(X.dot(ws))) - y*(X.dot(ws)))
        
        R2 = 1-(loglike/loglike0)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break

-------------------------------------------------
Feature chosen:  DER_lep_eta_centrality (index : 12 )
-------------------------------------------------
Feature chosen:  PRI_tau_pt (index : 13 )
-------------------------------------------------
Feature chosen:  DER_mass_transverse_met_lep (index : 1 )
-------------------------------------------------
Feature chosen:  DER_met_phi_centrality (index : 11 )
-------------------------------------------------
Feature chosen:  PRI_jet_num (index : 22 )
-------------------------------------------------
Feature chosen:  DER_deltaeta_jet_jet (index : 4 )
-------------------------------------------------
Feature chosen:  PRI_met (index : 19 )
-------------------------------------------------
Feature chosen:  PRI_jet_all_pt (index : 29 )
-------------------------------------------------
Feature chosen:  PRI_lep_pt (index : 16 )
-------------------------------------------------
Feature chosen:  DER_pt_h (index : 3 )
--------------------------------

In [18]:
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

R2 asjusted values:
0.19737696412
0.269403358329
0.353843302574
0.377653976523
0.413845668945
0.433729784756
0.438987198133
0.449027452605
0.462165599802
0.477094977648
0.482883945115
0.48696066974
0.498932606922
0.500747961218
0.502078522523
0.502863518365
0.502913791686
0.502954234765
0.50298559172
0.502998809377
0.503001618212
0.50300410961
-------------------------------------------------------
Number of features chosen: 22


Indices of features chosen:  [12, 13, 1, 11, 22, 4, 19, 29, 16, 3, 23, 2, 7, 6, 21, 26, 18, 20, 9, 28, 15, 24]


### Ridge regression with cross validation (needed)

I used only lambda as hyperparameter because I am not building a polynomial model. But I added different features transformation (square or log or power), so maybe we can test the polynomial after the best features are selected 

#### Using the loss of ridge regression as the error of R2

In [19]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))


In [20]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
k = 0 #needed for the R^2 adjusted

w0, loss = ridge_regression(Y,X,0)  # start with lambda = 0  
y = predict_labels(w0, X)

sse = loss
sst = np.sum((Y - Y.mean())**2)  #lack of information
R2 = np.abs((sst-sse)/sst)
R2adj_0 = R2 - (k/(n-k-1)*(1-R2))

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        #CROSS VALIDATION
        w_tr_tot, loss_tr_tot, loss_te_tot = cross_validation_rr(Y,X)
        
        ws = w_tr_tot[np.argmin(loss_te_tot)]
        loss = np.min(loss_te_tot)
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        #y = predict_labels(ws,X)   #***** NO USE OF PREDICTION
        SSE = np.sum(loss**2)
        SST = np.sum((Y- Y.mean())**2)
        R2 = np.abs((SST-SSE)/SST)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1], '(index :', features[ind_max][0], ')')
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break
        

ValueError: setting an array element with a sequence.